# BigQuery を活用したビッグデータ分析
- Jupyter Notebook を用いたデータ分析を行います
- Pythonのデータ解析ライブラリであるPandasを用います
- ペタバイト規模のデータ分析が可能なBigQuery を活用することで、大規模なデータをストレスなくPython で分析することができます

## Install liblaries
- Notebook 起動後、一度だけ実行してください

In [5]:
!pip install --upgrade google
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery-storage
!pip install pandas
!pip install db_dtypes

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
 

- - -

## BigQuery のPython クライアントライブラリを用いて、BigQuery へクエリを実行する

[Python Client for Google BigQuery](https://cloud.google.com/python/docs/reference/bigquery/latest "Python Client for Google BigQuery")

Python から BigQuery を操作することができるClient Library



In [1]:
from google.cloud import bigquery
import pandas as pd
import db_dtypes

以下を置き換えてください
- project_id : 自身のProject ID
- dataset_name : 先ほど作成したBigQuery のデータセット名（名前_dsd_dataset）

In [20]:
project_id='hclsj-datascience-day'
dataset_name='keihoshino_dsd_dataset'

データセットの中のテーブルの一覧を取得する

In [6]:

client = bigquery.Client()
print("Tables:")
for t in list(client.list_tables(project_id+'.'+dataset_name)):
  print(t.table_id)

Tables:
sample_table


先ほどBigQuery で実行したクエリを実行し、Pandas Dataframe に格納する

<details><summary>Pandasとは</summary>
Pandasは、データ分析や操作のためのオープンソースのPythonライブラリです。<br>大量のデータを効率的に処理するために、データフレームと呼ばれる構造を提供します。<br>データフレームは、行と列を持つ表形式のデータ構造で、さまざまなデータ型 (数値、文字列、日付など) を扱うことができます。

主な機能：
- ファイル読み込み：CSV, Excel, SQLデータベースなど
- データ操作：フィルタリング、並べ替え、集計、結合
- データ分析：統計、時系列分析、データ変換
</details>

In [9]:
sql = '''
SELECT
  Id,
  SUM(TotalSteps) AS SumTotalSteps, 
  SUM(TotalDistance) AS SumTotalDistance,
  SUM(Calories) AS SumCalories
FROM `hclsj-datascience-day.keihoshino_dsd_dataset.sample_table`
WHERE FORMAT_DATE("%Y", ActivityDate) = "2016"
AND FORMAT_DATE("%m", ActivityDate) = "04"
GROUP BY Id
ORDER BY SumCalories DESC
Limit 100
'''

In [11]:
df_sample_query_result = client.query(sql).to_dataframe() 

結果を見る<br>（1〜10行目のみ表示する）

In [12]:
df_sample_query_result[0:10]

,Id,SumTotalSteps,SumTotalDistance,SumCalories
0,5577150313,183247,137.119999,69399
1,8877689391,333267,274.799999,68480
2,8378563200,184488,146.310003,66798
3,8053475328,309238,240.519999,58771
4,4388161847,192846,148.950001,58454
5,4702921684,166683,135.220000,57601
6,1644430081,142055,103.270001,54252
7,8583815059,134148,104.649999,53179
8,7007744171,234789,165.829999,51138
9,6290855005,103392,78.190000,50960


- - - 

## BigQuery のマジックコマンドを用いて、BigQuery へクエリを実行し、Pandas Dataframe へ格納する

[BigQuery マジックコマンドについて](https://cloud.google.com/bigquery/docs/visualize-jupyter?hl=ja#query_notebook_data_using_the_bigquery_magic_command)

Jupyter NotebookでBigQueryへクエリを実行する際、%%bigqueryというマジックコマンドを使用できます。<br>このマジックコマンドは、Jupyterセル内でBigQueryへの接続、クエリ実行、結果の取得などを簡単に行えるようにします。<br>Jupyter Notebook からクエリをする際はこの方法が簡単なのでオススメです！

BQ Magic Commend 使い方

```
%%bigquery DATAFRAME名
SELECT * FROM YOUR_TABLE_NAME
LIMIT 10;
```

オプション:<br>
--project: 使用するGCPプロジェクト名<br>
--dataset: データセットの名前<br>
--format: 結果のフォーマット (デフォルトはpandas DataFrame)<br>
--verbose: 詳細な実行ログを表示<br>

マジックコマンドの有効化

In [13]:
%load_ext google.cloud.bigquery

クエリの実行

In [15]:
%%bigquery df_magic
SELECT
  Id,
  SUM(TotalSteps) AS SumTotalSteps, 
  SUM(TotalDistance) AS SumTotalDistance,
  SUM(Calories) AS SumCalories
FROM `hclsj-datascience-day.keihoshino_dsd_dataset.sample_table`
WHERE FORMAT_DATE("%Y", ActivityDate) = "2016"
AND FORMAT_DATE("%m", ActivityDate) = "04"
GROUP BY Id
ORDER BY SumCalories DESC
Limit 100

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/job/query.py:1859: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2153: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


結果の表示

In [16]:
df_magic[0:10]

,Id,SumTotalSteps,SumTotalDistance,SumCalories
0,5577150313,183247,137.119999,69399
1,8877689391,333267,274.799999,68480
2,8378563200,184488,146.310003,66798
3,8053475328,309238,240.519999,58771
4,4388161847,192846,148.950001,58454
5,4702921684,166683,135.220000,57601
6,1644430081,142055,103.270001,54252
7,8583815059,134148,104.649999,53179
8,7007744171,234789,165.829999,51138
9,6290855005,103392,78.190000,50960


テーブルのスキーマ情報を取得する

In [23]:
%%bigquery
SELECT column_name
FROM `hclsj-datascience-day.keihoshino_dsd_dataset.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = table_name

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/job/query.py:1859: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2153: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,column_name
0,Id
1,ActivityDate
2,TotalSteps
3,TotalDistance
4,TrackerDistance
5,LoggedActivitiesDistance
6,VeryActiveDistance
7,ModeratelyActiveDistance
8,LightActiveDistance
9,SedentaryActiveDistance


- - - 

## Finish!!